In [7]:
import pandas as pd

In [8]:
data = pd.read_csv('../data/test.csv')
data = data[['text', 'Label']]
data.head()

,text,Label
0,Новая версия Kandinsky от Сбера научилась дела...,Технологии
1,"🌩️В Подмосковье молния чуть не убила людей, ко...",Технологии
2,"Какая она, настоящая француженка? «‎Сильная, н...",Развлечения
3,▫️ **Выходит крепкий обзор** ситуации на росси...,Образовательный контент
4,❗️Глава МАГАТЭ Гросси по итогам осмотра Запоро...,Технологии


In [9]:
data['Label'].value_counts()

Label
Технологии                 145
Политика                   105
Развлечения                 45
Образовательный контент     44
Fashion                     33
Финансы                     32
Общее                       20
Путешествия/релокация       15
Шоубиз                      12
Крипта                       4
Name: count, dtype: int64

In [10]:
data['text'].loc[data['Label']=="Финансы"]

5      Во втором микроавтобусе Пригожина нашли ещё 10...
9      По предварительным оценкам, в Белгородской обл...
10             ⭕продал SNGSP по 41.175 RUB\nInvest_doxod
26     Научный факт: мы привыкли к бесконтактной опла...
38     🗣 **Предлагаю выделить дополнительной выделить...
45     **Совкомбанк Лизинг запускает в июле AI-платфо...
67     ЦБ России не видит рисков финансовой стабильно...
119    ❓Почему доллар обречен на провал, а при этом р...
121    Росстат: потребительские цены в РФ с начала 20...
157    **Новая зацепка в деле «Северных потоков» гово...
165    Около двух месяцев назад на встрече со знакомы...
174    Въезд россиян на личном транспорте в Германию ...
180    🇷🇺 По словам президента России Владимира Путин...
222    **Метро на Виноградарь в 2024 году не построят...
231    Внешний долг России упал до минимума с 2006 го...
245    **Шоколад будет стоить дорого. Все дело в цена...
247    Трейдинг: **Если следовать сезонности рынка + ...
254    😏** Продать больше нефти

In [11]:
data.iloc[0]

text     Новая версия Kandinsky от Сбера научилась дела...
Label                                           Технологии
Name: 0, dtype: object

In [12]:
post_themes = ['Финансы', 'Технологии', 'Политика',
                   'Шоубиз', 'Fashion', 'Крипта', 'Путешествия/релокация',
                   'Образовательный контент', 'Развлечения', 'Общее']

storage = {}

In [13]:
for post_theme in post_themes:
    posts = data['text'].loc[data['Label']==post_theme].to_list()
    storage[post_theme] = posts[:2]

storage

{'Финансы': ['Во втором микроавтобусе Пригожина нашли ещё 10 миллиардов рублей — они лежали в 80 коробках из под обуви. В каждой коробочке по 75 миллионов рублей.',
  'По предварительным оценкам, в Белгородской области понадобится 5 млрд руб на восстановление жилья, более 5 млрд на восстановление учреждений соцсферы, еще 10 млрд – на восстановление предприятий, сообщил Гладков.\n\n🟩 [Подпишись — RT на русском](https://t.me/+UnkqFPSZsedkOTZi)'],
 'Технологии': ['Новая версия Kandinsky от Сбера научилась делать еще более высококачественные изображения. Нейросеть значительно улучшила свои показатели — с её помощью можно делать как футуристичные арты, так и реалистичные картинки. \n\nТакже в нейросети добавили новые функции. Точная дата релиза обновления неизвестна, но обещают, что это произойдет уже совсем скоро. \n\n@nuancesprog',
  '🌩️В Подмосковье молния чуть не убила людей, которые прятались от ливня под зонтом — он стал громоотводом'],
 'Политика': ['[Памфилова:](https://t.me/rusbrie

In [14]:
import numpy as np
import torch.nn.functional as F

from torch import Tensor
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained('intfloat/multilingual-e5-base')
model = AutoModel.from_pretrained('intfloat/multilingual-e5-base')


def average_pool(last_hidden_states: Tensor,
                 attention_mask: Tensor) -> Tensor:
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]


# Each input text should start with "query: " or "passage: ", even for non-English texts.
# For tasks other than retrieval, you can simply use the "query: " prefix.
for post_theme in post_themes:
    posts = data['text'].loc[data['Label']==post_theme].to_list()[2:]
    for post in posts:
        input_texts = ['query' + storage[post_theme][0], 
                       'query' + storage[post_theme][1], 
                       'query' + post]

        # Tokenize the input texts
        batch_dict = tokenizer(input_texts, max_length=512, padding=True, truncation=True, return_tensors='pt')

        outputs = model(**batch_dict)
        embeddings = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])

        # normalize embeddings
        embeddings = F.normalize(embeddings, p=2, dim=1)
        scores = (embeddings[:2] @ embeddings[2:].T) * 100
        print('1)' + storage[post_theme][0], '2)' + storage[post_theme][1], '3)' + post, sep='\n')
        print(scores.tolist())
        scores = np.array(scores.tolist()).flatten()
        max_similarity_indx = np.argmax(scores)
        if scores[max_similarity_indx] > 90:
            if len(post) > len(storage[post_theme][max_similarity_indx]):
                storage[post_theme][max_similarity_indx] = post
                print("ПРОИЗОШЛА ЗАМЕНА")
            print('-' * 100)


1)Во втором микроавтобусе Пригожина нашли ещё 10 миллиардов рублей — они лежали в 80 коробках из под обуви. В каждой коробочке по 75 миллионов рублей.
2)По предварительным оценкам, в Белгородской области понадобится 5 млрд руб на восстановление жилья, более 5 млрд на восстановление учреждений соцсферы, еще 10 млрд – на восстановление предприятий, сообщил Гладков.

🟩 [Подпишись — RT на русском](https://t.me/+UnkqFPSZsedkOTZi)
3)⭕продал SNGSP по 41.175 RUB
Invest_doxod
[[80.153564453125], [78.71159362792969]]
1)Во втором микроавтобусе Пригожина нашли ещё 10 миллиардов рублей — они лежали в 80 коробках из под обуви. В каждой коробочке по 75 миллионов рублей.
2)По предварительным оценкам, в Белгородской области понадобится 5 млрд руб на восстановление жилья, более 5 млрд на восстановление учреждений соцсферы, еще 10 млрд – на восстановление предприятий, сообщил Гладков.

🟩 [Подпишись — RT на русском](https://t.me/+UnkqFPSZsedkOTZi)
3)Научный факт: мы привыкли к бесконтактной оплате ✨

Нет 

In [15]:
import os 
import sys 

try:
    path = os.path.abspath('../models/')
    sys.path.append(path)
    from sentence_similarity import compute_similarity
except Exception as e:
    print(e)


In [16]:
for post_theme in post_themes:
    posts = data['text'].loc[data['Label']==post_theme].to_list()[2:]
    for post in posts:
        input_posts = [post for post in storage[post_theme]] + [post]
        scores = compute_similarity(input_posts)
        for indx, input_post in enumerate(storage[post_theme] + [post]):
            print(f'{indx + 1}) {input_post}')
        print(scores)
        print('-'*100)



1) Во втором микроавтобусе Пригожина нашли ещё 10 миллиардов рублей — они лежали в 80 коробках из под обуви. В каждой коробочке по 75 миллионов рублей.
2) По предварительным оценкам, в Белгородской области понадобится 5 млрд руб на восстановление жилья, более 5 млрд на восстановление учреждений соцсферы, еще 10 млрд – на восстановление предприятий, сообщил Гладков.

🟩 [Подпишись — RT на русском](https://t.me/+UnkqFPSZsedkOTZi)
3) ⭕продал SNGSP по 41.175 RUB
Invest_doxod
[80.15356445 78.71159363]
----------------------------------------------------------------------------------------------------
1) Во втором микроавтобусе Пригожина нашли ещё 10 миллиардов рублей — они лежали в 80 коробках из под обуви. В каждой коробочке по 75 миллионов рублей.
2) По предварительным оценкам, в Белгородской области понадобится 5 млрд руб на восстановление жилья, более 5 млрд на восстановление учреждений соцсферы, еще 10 млрд – на восстановление предприятий, сообщил Гладков.

🟩 [Подпишись — RT на русском]